In [1]:
#holay molay :0 ヾ(＠⌒ー⌒＠)ノ  ╚(•⌂•)╝   (≧∀≦)ゞ
#finish:
#min p add ✓
#fix sample nexttoken ✓
#logtoku detection
###adapt get_eu for per-token basis ✓
###implement R_token = -AU_token * EU_token ✓
###logtoku fucntion(create sliding window evaluating r_response = (1/K) Σ R(aₜ) )
#verify generation loop works ✓
#verify hallucination detection works (⊙_⊙;)
!pip install transformers
!pip install scipy
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 78.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 135.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 147.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 132.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.2 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.6/199.6 kB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 160.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.4.0
    Uninstalling fsspec-

In [2]:
import numpy as np
import scipy
from scipy.special import softmax, digamma
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import gc

#chatgpt said this would get rid of variation when greedy decoding
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

NameError: name 'random' is not defined

In [46]:
#topk, 
#get_uncertainty_metric (renamed from get_eu), 
#get_one_pass_metric for logtoku pasted from their codebase

def topk(arr, k):
    indices = np.argpartition(arr, -k)[-k:]
    values = arr[indices]
    return values, indices
    
def get_uncertainty_metric(logits, mode="eu", k=25):
    if mode == "eu":
        if k is None:
            raise ValueError("k must be provided for 'eu' mode.")

        def eu(logits):
            top_k = k
            if len(logits) < top_k:
                raise ValueError(f"Logits array length ({len(logits)}) is less than top_k (= {k}). \nLogits shape: {logits.shape}, first 10 logits: {logits.squeeze(0).cpu().numpy()[:10]}")
            top_values, _ = topk(logits, top_k)
            mean_scores = top_k / (np.sum(np.maximum(0, top_values)) + top_k)
            return mean_scores
        return eu

    
    elif mode == "au":
        def cal_au(logits):
            top_k = k
            if len(logits) < top_k:
                raise ValueError("Logits array length is less than top_k.")
            top_values = np.partition(logits, -top_k)[-top_k:]
            alpha = np.array([top_values])
            alpha_0 = alpha.sum(axis=1, keepdims=True)
            psi_alpha_k_plus_1 = digamma(alpha + 1)
            psi_alpha_0_plus_1 = digamma(alpha_0 + 1)
            result = - (alpha / alpha_0) * (psi_alpha_k_plus_1 - psi_alpha_0_plus_1)
            print(f"k = {k}")
            return result.sum(axis=1)[0]
        return cal_au
    
    else:
        raise ValueError(f"Unsupported mode: {mode}")

def get_token_reliability(logits, k=25):
    au_fn = get_uncertainty_metric(logits, mode= "au", k=k)
    eu_fn = get_uncertainty_metric(logits, mode= "eu", k=k)   
    au = au_fn(logits)
    eu = eu_fn(logits)
    print(f"au: {au}, eu: {eu}, rel.: {-au * eu}")
    return -au * eu


#unused so far?
def get_one_pass_metric(logits, clean_generated_tokens_length, metrics, get_eu, topk):
    """
    Process logits, calculate various metrics, and extract topk information from the logits.
    
    Args:
        
        clean_generated_tokens_length (int): Length of the generated tokens.
        metrics (list): List of metrics and their parameters.
        get_eu (function): Function to retrieve metric values.
        topk (function): Function to extract topk values.
        
    Returns:
        tuple: A tuple containing metric_dict and logit_dict.
    """
    metric_dict = {}

    sequence_length = len(logits)
    for idx_l in range(min(clean_generated_tokens_length, sequence_length)):
        logit = logits[idx_l]
        logit = logit.cpu().numpy()
        
        for metric, k in metrics:
            if metric not in metric_dict:
                metric_dict[metric] = []
            eu = get_eu(metric, k)
            metric_dict[metric].append(eu(logit[0]))

    logit_dict = {}
    logit_start_idx = 0
    logit_end_idx = min(clean_generated_tokens_length, sequence_length)
    ii = 0

    # Extract topk information from logits
    for idx_ll in range(logit_start_idx, logit_end_idx):
        logit = logits[idx_ll]
        logit = logit.cpu().numpy()
        
        top_k = 10
        top_values, top_indices = topk(logit[0], top_k)
        
        logit_dict[ii] = {'top_values': top_values, 'top_indices': top_indices}
        ii += 1

    # Convert top_values and top_indices in logit_dict to list
    for key in logit_dict:
        logit_dict[key]['top_values'] = logit_dict[key]['top_values'].tolist()
        logit_dict[key]['top_indices'] = logit_dict[key]['top_indices'].tolist()

    return metric_dict, logit_dict

In [52]:
import torch
import gc

if "generator" in globals():
    if hasattr(generator, "model"):
        del generator.model
    if hasattr(generator, "tokenizer"):
        del generator.tokenizer

torch.cuda.empty_cache()
gc.collect()


class SemanticSpillway:
    def __init__(self, model_path: str = "../models/Llama-3.1-8B-Instruct"):
        self.device = torch.device("cuda")# if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_path, 
            torch_dtype=torch.float16, 
            local_files_only=True
        ).to(self.device)
        self.reliability_scores = [] #for logtoku

        #not necessary unless/until we do processing in batches(then have to do padding and attention masks and blah blah blah)
        # if self.tokenizer.pad_token is None:
        #     self.tokenizer.pad_token = self.tokenizer.eos_token

    # (currently) uses min p to sample next token
    def sample_next_token(self, logits, temperature=0.7, top_k=50, top_p=0.9, min_p = 0.05, min_tokens_to_keep = 5, do_sample=True, k=25):
        
        # Temperature
        logits = logits / temperature
        
        # # top-k 
        # if top_k > 0:
        #     top_k = min(top_k, logits.size(-1))
        #     indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        #     logits[indices_to_remove] = float('-inf')
        
        # # top-p 
        # if top_p < 1.0:
        #     sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        #     cumulative_probs = torch.cumsum(torch.softmax(sorted_logits, dim=-1), dim=-1)
        #     sorted_indices_to_remove = cumulative_probs > top_p
        #     sorted_indices_to_remove[..., 0] = 0
        #     indices_to_remove = sorted_indices_to_remove.scatter(-1, sorted_indices, sorted_indices_to_remove)
        #     logits[indices_to_remove] = float('-inf')

        
        # min-p truncation: gets rid of tokens w/ probabilities less than min_p * max_prob (ie probability of most likely token)
        # pasted from paper
        # turn into methods later i dislike this block of code
        # Convert logits to probabilities
        probs = torch.softmax(logits, dim=-1)
        # Get the probability of the top token for each sequence in the batch
        top_probs, _ = probs.max(dim=-1, keepdim=True)
        # Calculate the actual min_p threshold by scaling min_p with the top token's probability
        scaled_min_p = min_p * top_probs
        # Create a mask for tokens that have a probability less than the scaled min_p
        tokens_to_remove = probs < scaled_min_p

        sorted_indices = torch.argsort(logits, descending=True, dim=-1)
        sorted_indices_to_remove = torch.gather(tokens_to_remove, dim=-1, index=sorted_indices)
        # Keep at least min_tokens_to_keep
        sorted_indices_to_remove[..., : min_tokens_to_keep] = False

        indices_to_remove = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
        filter_value = float('-inf') #TODO verify that this is the correct value to use
        scores_processed = logits.masked_fill(indices_to_remove, filter_value)
        
        # sample or greedy
        if do_sample:
            probs = torch.softmax(scores_processed, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
        else:
            next_token = torch.argmax(scores_processed, dim=-1, keepdim=True)
            #next_token = torch.argmax(logits, dim=-1).unsqueeze(0)
        return next_token

    # main generation loop w/ stopping control
    # halu_detect = which function to use for hallucination control. currently only option is logtoku, but for regeneration, ner + logtoku function will be created to use instead
    def generation_loop(self, prompt, max_new_tokens=100, temperature=1.0, top_k=50, top_p=0.9, min_p = 0.05, 
                 k=25, do_sample=True, halu_detect=None):

        
        self.model.eval()
        
        #turn text into token ids
        input_ids = self.tokenizer.encode(prompt, return_tensors="pt").to(self.device)
        #input_ids = input_ids.long()
        
        for step in range(max_new_tokens):
            with torch.no_grad(): #.no_grad to ignore gradients for inference
                
                outputs = self.model(input_ids)
                logits = outputs.logits[:, -1, :]
                
                # sample next token
                next_token = self.sample_next_token(logits, temperature,  do_sample)
                
                # # check for end of sequence -- if it is, end loop
                if next_token == self.tokenizer.eos_token_id:
                    print("found an eos")
                    break
                #or
                if next_token in self.tokenizer.encode("<|endoftext|>"):
                    print("found an eos")
                    break
                
                # add new token to sequence
                input_ids = torch.cat([input_ids, next_token], dim=-1) 
                
                # callback fn to implement hallucination detection for stopping generation
                # if fn exists and returns false (aka hallucination is detected), stop generating 
                logits_1d = logits.squeeze(0).cpu().numpy()
                if halu_detect and not halu_detect(step, self.reliability_scores, logits_1d, k):
                    break
                    
        #turn token ids back into clean text
        return self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
        

# returns true if no hallucination detected (clean), returns false if not
# used as callback in greater generation loop (intended for initial generation)
def logtoku(step, reliability_scores, logits, k=25):
    tok_rel = get_token_reliability(logits, k)
    #print(f"token reliability: {tok_rel}")
    # if step >=20:
    #     print("Hallucination detected... ceasing generation.")
    #     return False
    return True

# returns true if no hallucination detected (clean), returns false if not
# used as callback in greater generation loop (intended for comparing with retrieved context)
def ner_based():
    #empty function to fill in later
    return True

#combining the above methods to use at once?!（⊙ｏ⊙）
def ner_logtoku():
    #empty
    return True



generator = SemanticSpillway("../models/Llama-3.1-8B-Instruct")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [14]:
import torch
import gc

# Check current GPU memory usage
print(f"GPU memory allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"GPU memory cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
print(f"GPU memory available: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

# See what's taking up memory
torch.cuda.memory_summary()

GPU memory allocated: 14.97 GB
GPU memory cached: 15.11 GB
GPU memory available: 23.54 GB


'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  15333 MiB |  15492 MiB |   1312 GiB |   1297 GiB |\n|       from large pool |  15330 MiB |  15436 MiB |   1067 GiB |   1052 GiB |\n|       from small pool |      3 MiB |     57 MiB |    244 GiB |    244 GiB |\n|---------------------------------------------------------------------------|\n| Active memory         |  15333 MiB |  15492 MiB |   1312 GiB |   1297 GiB |\n|       from large pool |  15330 MiB |  15436 MiB |

In [59]:
# test generation
result = generator.generation_loop("Question: What is the capital of France? Answer:  ", max_new_tokens=200, temperature=0.8, do_sample=False)
print(result)

Question: What is the capital of France? Answer:   - 
A. Berlin
B. Paris
C. Madrid
D. Rome
Answer: B. Paris
Explanation: Paris is the capital city of France. It is a major center of culture, fashion, and learning, and is home to the famous Louvre Museum and many other historical landmarks. Paris has been a major city for over two thousand years and is a popular destination for tourists and business travelers alike. It is located in the northern part of France, near the city of Versailles and the Seine River. Paris is known for its beautiful architecture, art, and cuisine, and is considered one of the most romantic cities in the world. The city has a population of over 2.1 million people and is the center of the Île-de-France region. The name "Paris" is derived from the ancient Celtic tribe known as the Parisii, who inhabited the area around 250 BC. The city has been ruled by various civilizations throughout its history, including the


In [55]:
# test generation with question to trigger hallucination + logtoku detection
result = generator.generation_loop(
    "Answer the question concisely and accurately. Q: What were the main outcomes of the 1847 Treaty of Millbrook between the United States and the Cherokee Nation? A:", #this doesn't exist btw lol
    max_new_tokens=120,
    halu_detect=logtoku,
    k=7,
    temperature=0.7
)
print(result)

result = generator.generation_loop(
    "Answer the question concisely and accurately. Q: What is the capital of France? A:",
    max_new_tokens=120,
    halu_detect=logtoku,
    k=7,
    temperature = 0.7
)
print(result)

k = 7
au: 1.9106452525172894, eu: 0.06722689075630252, rel.: -0.12844673966502787
k = 7
au: 1.9109331796550468, eu: 0.06096897114861187, rel.: -0.1165076298973137
k = 7
au: 1.9072790432137179, eu: 0.06018269747447609, rel.: -0.11478519765713939
k = 7
au: 1.91355936389064, eu: 0.05676634566649772, rel.: -0.10862577230397957
k = 7
au: 1.9109155755243572, eu: 0.057318321392016376, rel.: -0.10953047311091504
k = 7
au: 1.9073309458369154, eu: 0.0619126589275843, rel.: -0.11808793031162772
k = 7
au: 1.9095557315726175, eu: 0.058885383806519455, rel.: -0.11244492215359263
k = 7
au: 1.9085124584650115, eu: 0.06243032329988852, rel.: -0.11914904980383573
k = 7
au: 1.9081736639779183, eu: 0.06130268199233716, rel.: -0.11697616330899116
k = 7
au: 1.919819332219023, eu: 0.05539070227497527, rel.: -0.10634014105268574
k = 7
au: 1.9129062809060513, eu: 0.059989287627209426, rel.: -0.11475388508916858
k = 7
au: 1.9100262509487758, eu: 0.06031233171782445, rel.: -0.11519813683697518
k = 7
au: 1.899411